#### Install relevant libraries

In [1]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference youtube-transcript-api pytube sentence-transformers

In [2]:
!pip install selenium
!pip install webdriver_manager
!apt-get install -y unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 17.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
--2024-08-11 21:48:44--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.201.190, 74.125.201.91, 74.125.201.93, ...
Connecting to dl.google.com (dl.google.com)|74.125.201.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109464716 (104M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 104.39M   237MB/s    in 0.4s    

2024-08-11 21:48:45 (237 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [109464716/109464716]

Selecting previo

# Web Scrape

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from langchain.schema import Document

In [4]:
url = "https://en.wikipedia.org/wiki/Air_Jordan"
options = webdriver.ChromeOptions()
# options.binary_location = "/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome"

options.add_argument('--headless')  # Headless mode for running without UI
options.add_argument('--no-sandbox') # Bypass sandboxing - can be a security risk, use with caution
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the webpage
driver.get(url)

# Wait for the page to load completely (You can use WebDriverWait for more complex scenarios)
driver.implicitly_wait(10)

# Extract the full page content after JavaScript execution
content = driver.page_source

# Close the driver
driver.quit()

# Parse the content with BeautifulSoup (optional, for further processing)

soup = BeautifulSoup(content, 'html.parser')

title = soup.find('title').text if soup.find('title') else 'No title found'
description = soup.find('meta', attrs={'name': 'description'})
description_content = description['content'] if description else 'No description found'

keywords = soup.find('meta', attrs={'name': 'keywords'})
keywords_content = keywords['content'] if keywords else 'No keywords found'

metadata = {
    'source': url,
    'title': title,
    'description': description_content,
    'keywords': keywords_content
}

# Extract all paragraphs or desired content
paragraphs = soup.find_all('p')
page_content = "\n".join([p.get_text() for p in paragraphs])

# data = [Document(metadata=metadata, page_content=page_content)]
# data = [dict(metadata=metadata, page_content=page_content)]

In [5]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
import os

pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# openai_api_key = userdata.get("OPENAI_API_KEY")
# os.environ['OPENAI_API_KEY'] = openai_api_key

# Initialize Pinecone:



In [6]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("sneaker-bot")

index_name = "sneaker-bot"
namespace = "jordan"

# Initialize Hugging Face

In [7]:
# HuggingFace Embeddings
# Use this instead of OpenAI embeddings if you don't have an OpenAI account with credits

# hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# hf_embeddings = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-Qwen1.5-7B-instruct")
# hf_embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-mistral-7b-instruct")

hf_embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

#Test Embedding

In [8]:
text = "This is a test document."
query_result = hf_embeddings.embed_query(text)
print(f"Embedding dimension:{len(query_result)}")

Embedding dimension:1024


#Dynamic text splitter

In [14]:
import nltk
from langchain.schema import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

# Ensure the necessary NLTK data is downloaded
nltk.download('punkt')

# Initialize the tokenizer
tokenizer = tiktoken.get_encoding('p50k_base')

# Custom length function based on tokenization
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

# Dynamic chunking function
def dynamic_chunking(text, max_chunk_size=50, overlap_size=5):
    sentences = nltk.sent_tokenize(text)  # Split text into sentences
    chunks = []
    current_chunk = []
    current_chunk_len = 0

    for sentence in sentences:
        sentence_len = tiktoken_len(sentence)
        if current_chunk_len + sentence_len > max_chunk_size:
            # If the current chunk is full, add it to the chunks list
            chunks.append(" ".join(current_chunk))
            # Start a new chunk with an overlap from the previous chunk
            current_chunk = current_chunk[-overlap_size:]
            current_chunk_len = tiktoken_len(" ".join(current_chunk))

        current_chunk.append(sentence)
        current_chunk_len += sentence_len

    # Add the last chunk if it contains any sentences
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Chunk the text dynamically
chunks = dynamic_chunking(page_content)

# Create Document objects with each chunk
# documents = [Document(metadata=metadata, page_content=chunk) for chunk in chunks]
documents = [Document(page_content=chunk) for chunk in chunks]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
print(f"No of chunks:{len(documents)}")

No of chunks:191


# Upsert to Pinecone index

In [ ]:
vectorstore_from_texts = PineconeVectorStore.from_texts([f"{d.page_content}" for d in documents], hf_embeddings, index_name=index_name, namespace=namespace)

# Perform RAG

In [42]:
query = "What models of Air Jordan have been released?"

# x = pc.inference.embed(
#     "multilingual-e5-large",
#     inputs=[query],
#     parameters={
#         "input_type": "query"
#     }
# )
query_embedding = hf_embeddings.embed_query(query)

top_matches = pinecone_index.query(
    namespace=namespace,
    vector=query_embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

In [ ]:
top_matches

In [ ]:
# Get the list of retrieved texts
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
contexts

#Text Summarization (GPT-2)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "openai-community/gpt2-medium"  # Model name on Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
primer = f"""You are a personal assistant. Answer any questions about Jordan.
"""
augmented_query = "<CONTEXT>\n" + "".join(contexts[ : 3]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query
input_text = primer + "\n" + augmented_query

# Tokenize the input
inputs = tokenizer.encode(input_text, return_tensors="pt")

In [ ]:
outputs = model.generate(inputs, max_length=300, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

# Decode the generated output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

# Relevant code for Sneaker Bot ends above this line

#Intialize open router

In [ ]:
# Free Llama 3.1 API via OpenRouter
# Use this instead of OpenAI if you don't have an OpenAI account with credits

openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get("OPENROUTER_API_KEY")
)

## Initialize our text splitter (Another way)


This is how we will chunk up the text to be retrieved during the RAG process

In [ ]:
tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=5,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
)

# Understanding Embeddings

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    # Call the OpenAI API to get the embedding for the text
    response = openai_client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

def cosine_similarity_between_words(sentence1, sentence2):
    # Get embeddings for both words
    embedding1 = np.array(get_embedding(sentence1))
    embedding2 = np.array(get_embedding(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    print("Embedding for Sentence 1:", embedding1)
    print("\nEmbedding for Sentence 2:", embedding2)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like walking to the office"


similarity = cosine_similarity_between_words(sentence1, sentence2)
print(f"\n\nCosine similarity between '{sentence1}' and '{sentence2}': {similarity:.4f}")


In [ ]:
texts = text_splitter.split_documents(data)

In [ ]:
embeddings.data[0].values

#Data in Pinecone

Documentation: https://docs.pinecone.io/integrations/langchain#key-concepts

In [ ]:
for document in documents:
    print("\n\n\n\n----")

    print(document.metadata, document.page_content)

    print('\n\n\n\n----')

In [ ]:
vectorstore_from_texts = PineconeVectorStore.from_texts([f"Source: {t.metadata['source']}, Title: {t.metadata['title']} \n\nContent: {t.page_content}" for t in texts], embeddings, index_name=index_name, namespace=namespace)

# Perform RAG (For open AI client)

In [ ]:
raw_query_embedding = openai_client.embeddings.create(
    input=[query],
    model="text-embedding-3-small"
)

query_embedding = raw_query_embedding.data[0].embedding

In [ ]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
print(augmented_query)

In [ ]:
# Modify the prompt below as need to improve the response quality

primer = f"""You are a personal assistant. Answer any questions I have about the Youtube Video provided.
"""

res = openai_client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

openai_answer = res.choices[0].message.content

In [ ]:
print(openai_answer)

# Using OpenRouter

In [ ]:
 # Check out different models here: https://openrouter.ai/docs/models

res = openrouter_client.chat.completions.create(
    model="openai-community/gpt2",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

answer = res.choices[0].message.content

In [ ]:
print(answer)

Pre-training is mentioned to be important as it establishes a base level of intelligence and common sense that can be further built upon through post-training techniques. Aravind discusses various aspects of pre-training, including the role of data, data quality, and token size, and how they contribute to the overall performance of language models.


# Putting it all together

In [ ]:
def perform_rag(query):
    raw_query_embedding = openai_client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    )

    query_embedding = raw_query_embedding.data[0].embedding

    top_matches = pinecone_index.query(vector=query_embedding, top_k=10, include_metadata=True, namespace=namespace)

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are an expert personal assistant. Answer any questions I have about the Youtube Video provided. You always answer questions based only on the context that you have been provided.
    """

    res = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return res.choices[0].message.content


In [ ]:
perform_rag("What does Aravind mention about pre-training and why it is important?")

"Aravind Srinivas discusses the importance of pre-training in the development of effective AI models, highlighting its role in creating a foundation of general common sense that is crucial for the model's performance. Here are the key points he mentions about pre-training and its significance:\n\n1. **Foundational Stage**: Pre-training is the stage where the raw scaling on compute happens. It involves training the model on a vast amount of data to develop a general understanding of language and common sense.\n\n2. **General Common Sense**: Without substantial pre-training, the model lacks the baseline common sense necessary for effective reasoning. This foundational knowledge is critical because it equips the model with a broad understanding of language and concepts.\n\n3. **Importance in Combination with Post-Training**: While post-training (which includes supervised fine-tuning and reinforcement learning from human feedback, or RLHF) is essential for refining and controlling the mode

In [ ]:
perform_rag("What advantages does Perplexity have over other AI companies?")

"Perplexity differentiates itself from other AI companies by focusing on a few unique aspects:\n\n1. **Answer-Centric Approach**: Unlike traditional search engines that display a list of URLs, Perplexity aims to provide direct, Wikipedia-like responses to queries. This method prioritizes giving users direct answers and relevant information over sending them to another webpage. This shifts the UI focus from a list of links to summarized answers, aiming to provide a more streamlined and valuable user experience.\n\n2. **Factual Grounding (RAG - Retrieval-Augmented Generation)**: Perplexity ensures their answers are factually grounded by only generating responses based on documents retrieved from the internet. This principle aims to reduce hallucinations by sticking closely to the retrieved content, enhancing the trustworthiness and accuracy of the information provided.\n\n3. **Knowledge-Centric Mission**: The company’s mission goes beyond search and aims to make people smarter by helping

# RAG over a PDF

In [ ]:
loader = PyPDFLoader("/content/Harry Potter and the Sorcerers Stone.pdf") # Insert the path to a PDF here
data = loader.load()

print(data)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100,
        length_function=tiktoken_len,
        separators=["\n\n", "\n", " ", ""]
    )

texts = text_splitter.split_documents(data)

# Insert all the chunks from the PDF into Pinecone
vectorstore_from_texts = PineconeVectorStore.from_texts([f"Source: {t.metadata['source']}, Title: {t.metadata['title']} \n\nContent: {t.page_content}" for t in texts], embeddings, index_name=index_name, namespace=namespace)

# After this, all the code is the same from the Perform RAG section of this notebook
# Since the data from the PDF is now stored in Pinecone, you can perform RAG over it the same way as the YouTube video